In the 'markdown cell' below  replace the `???` with the names of those in your group.

???

# Artificial neural networks for QSAR
_by David Holmberg (August 2023)_
#### Dataset
For this exercise we will use the same dataset of aqueous solubility of 1142 diverse chemical compounds as you previously explored during the QSAR lab last week. However, here we will only use the PhysChem descriptors.

#### Modelling comparisons
1. Compare the results of linear regression to those of a simple neural network with no hidden layers and no non-linear activation functions
2. Compare the results of a a random forest regressor, a support vector regressor, and a neural network with two hidden layers (with non-linear activations) and dropout.

#### Aims
* to see the link between neural networks and linear regression
* to learn the basics of how to define, compile, fit and evaluate neural networks via TensorFlow.

#### Note
We will be using the open-source machine learning framework TensorFlow (https://www.tensorflow.org) and Keras (https://keras.io) for our neural networks. TensorFlow was developed by the Google Brain team and is today one of the most widely used machine learning frameworks in research and industry and Keras was/is the most popular higher-level API that runs atop TensorFlow. However, last year TensorFlow 2 was released. In TensorFow 2 (which we will use for all our neural network work) Keras is now fully integrated. This means that we get all the benefits of TensorFlow with a much easier (Keras-type) way to define and train models than was previously possible with TensorFlow 1.

## Load packages

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Pytorch and Pytorch Geometric
import torch as tch
import torch.nn as nn
import torch.optim as optim

from torch_geometric.nn import GCNConv, summary as gsummary, global_mean_pool
from torch_geometric.data import Data, DataLoader

from rdkit import Chem
from rdkit.Chem import AllChem

# Helper libraries
from torchsummary import summary as asummary
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# tch.backends.cudnn.enabled = False
# tch.cuda.is_available = lambda : False
# device = tch.device('cpu')
!nvidia-smi | grep GeForce
device = tch.device("cuda" if tch.cuda.is_available() else "cpu")

## functions
Run these cells to have access to the necessary functions for the lab.

In [ ]:

def plot_history(train_losses, val_losses, model_name):
    fig = plt.figure(figsize=(15, 5), facecolor='w')
    ax = fig.add_subplot(121)
    ax.plot(train_losses)
    ax.plot(val_losses)
    ax.set(title=model_name + ': Model loss', ylabel='Loss', xlabel='Epoch')
    ax.legend(['Train', 'Test'], loc='upper right')
    ax = fig.add_subplot(122)
    ax.plot(np.log(train_losses))
    ax.plot(np.log(val_losses))
    ax.set(title=model_name + ': Log model loss', ylabel='Log loss', xlabel='Epoch')
    ax.legend(['Train', 'Test'], loc='upper right')
    plt.show()
    plt.close()



def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=128)
    return list(fp.ToBitString())

def smiles_to_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol

def read_smiles_data(path_data):
    df = pd.read_csv(path_data, sep=',')
    df['fingerprint'] = df['SMILES'].apply(smiles_to_fingerprint)
    df['fingerprint'] = df['fingerprint'].apply(lambda x: [int(bit) for bit in x])
    df['mol'] = df['SMILES'].apply(smiles_to_mol)
    return df

def make_pyg(row):
    # Create node features
    mol = row['mol']
    atom_features = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    x1 = tch.tensor(atom_features, dtype=tch.float).view(-1, 1)
    x2 = tch.tensor(row['fingerprint'], dtype=tch.float)
    y = tch.tensor(row['measured.log.solubility.mol.L.'], dtype=tch.float).view(-1, 1)
    # Duplicate fingerprint for each atom and concatenate
    x2_repeated = x2.repeat(x1.shape[0], 1)
    x = tch.cat([x1, x2_repeated], dim=1)
    
    # Create edge features (connectivity)
    edge_indices = []
    edge_features = []
    
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_indices.append((i, j))
        edge_features.append([bond.GetBondTypeAsDouble()])  # Bond type (as double)
    
    edge_index = tch.tensor(edge_indices, dtype=tch.long).t().contiguous()
    edge_attr = tch.tensor(edge_features, dtype=tch.float).view(-1, 1)
    
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)


## Load and preprocess data

#### Load and check shape of X and y

In [ ]:
df = read_smiles_data('data/solubility.csv')
df.head()

In [ ]:
X = np.load('data/X_qsar.npy')
y = np.load('data/y_qsar.npy')


data_pyg = df.apply(make_pyg, axis=1)
data_pyg = data_pyg[data_pyg.apply(lambda x: len(x.edge_index.shape) != 1)]
data_pyg.reset_index(drop=True, inplace=True)
dbl = int(len(data_pyg) * 0.3)
dbi = np.arange(dbl)
data_pyg= data_pyg[dbi[:dbl]]
data_pyg.reset_index(drop=True, inplace=True)
print(X.shape)
print(y.shape)

#### Split into training and test sets and standardize the data
Here we will just have a training and test set, so our results will not be quite as rigerous as those you got with cross-validation in the supervised machine learning lab.

In [ ]:
n_train = int(len(y) * 0.7) # 70% of data for training and 30% for testing

random.seed(1234)
indices = np.arange(len(y))
random.shuffle(indices)

# X_train0 is our training data prior to standardization
X_train0, X_test0 = X[indices[:n_train]], X[indices[n_train:]]
y_train, y_test = y[indices[:n_train]], y[indices[n_train:]]

# standardize X_train0 and X_test0 to give X_train and X_test
scaler = StandardScaler().fit(X_train0)
X_train = scaler.transform(X_train0)
X_test = scaler.transform(X_test0)


# Here we split the molecular data into a train and a test set
n_train = int(len(data_pyg) * 0.7) # 70% of data for training and 30% for testing
# random.seed(1234)
indices = np.arange(n_train)
# print(indices[-1])
data_train = data_pyg[indices[:n_train]]
data_train.reset_index(drop=True, inplace=True)
data_test = data_pyg[~data_pyg.isin(data_train)]
data_test.reset_index(drop=True, inplace=True)


## Linear Regression
## Random Forest Regressor & Support Vector Regressor
For comparative purposes, with the results we will explore later with a more involved neural network architectures than the one above, we will build a Linear Regression, Random Forest and Support Vector model. For these three machine learning algorithms we will just use the default hyper parameter settings, which are often a good place to start. This means that you will just have () after the model definition, as you did for the linear regression with LinearRegression(). To change the hyper parameters from the defaults one needs to specify them within the braces.

The code cells for the random forest and support vector regressors have been left blank below. You should fill in these cells. You should define the models, fit them, make predictions from them, compute their MSEs and print out the results.

* hint 1: look to the cell where we 'Load packages' to get the right model definition for the two machine learning methods
* hint 2: look at the cell with Linear Regression. It should be similar.

In [ ]:
#Linear Regression
LR_model = LinearRegression()
### Add Code Here
print('Linear Regression: MSE = ' + str(np.round(LR_mse, 3)))

#### Random Forest Regressor

In [ ]:
RF_model = RandomForestRegressor()
### Add Code Here
print('Random Forest Regressor: MSE = ' + str(np.round(RF_mse, 3)))

#### Support Vector Regressor

In [ ]:
SV_model = SVR()
### Add code here
print('Support Vector Regressor: MSE = ' + str(np.round(SV_mse, 3)))

## Artifical neural network as a linear regression
If we define a neural network with no hidden layers and no non-linear activations we essentailly get the same results as we do with basic linear regression. The results below should help clarify that to you (there are some minor differences hovever, hence the MSE for the neural network will not be _exactly_ the same as the results above for linear regression, but they are neverthelss very close).

<p>
    <img src="figs/lin-reg.png" alt="drawing" style="width:1200px;"/>
    <center>Figure 1. Our neural network version of linear regression.</center>
</p>

#### Define model
You will need to convert the data into tensors and define the model to use Neural Networks. IN this particular case, you'll get some help - but keep note of how it's done, you *will* have to do it yourselves in the rest of the assignment.

In [ ]:
X_train_tensor = tch.tensor(X_train, dtype=tch.float32).to(device)
y_train_tensor = tch.tensor(y_train, dtype=tch.float32).to(device).unsqueeze(1)  # Add dimension for regression
X_test_tensor = tch.tensor(X_test, dtype=tch.float32).to(device)
y_test_tensor = tch.tensor(y_test, dtype=tch.float32).to(device).unsqueeze(1)
# print(y_test_tensor)
# print(y_train_tensor)

In [ ]:

class ANN1(nn.Module):
    def __init__(self, input_dim):
        super(ANN1, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2 = nn.Linear(16, 1)  # Output layer with 1 nodes
        self.relu = nn.ReLU()

    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### Innitialize the model and set optimizer and learning rate hyperparameters
The learning rate and optimizer chosen below are both things that can be changed when one explores hyper parameter options, different architectures and what not. Below we use a learning rate (lr) of 0.001 (a common default learning rate) and the 'Adam' optimizer. take careful note here, you will be reimplementing this.

In [ ]:
input_dim = X_train.shape[1]
ann1_model = ANN1(input_dim).to(device).float()
asummary(ann1_model.to(device), input_size=(input_dim,))
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(ann1_model.parameters(), lr=0.001)  # Adjust learning rate as needed
num_epochs = 100


### Train the model
Here you will train the model. Take note of how the training loop looks. You will use similar loops for the rest of the assignment.

In [ ]:
num_epochs = 100
train_losses = []
val_losses = []
ann1_model.to(device)
pbar = tqdm(range(num_epochs), desc="Epochs")



for epoch in range(num_epochs):
    ann1_model.train()
    optimizer.zero_grad()
    outputs = ann1_model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())
    ann1_model.eval()
    with tch.no_grad():
        val_outputs = ann1_model(X_test_tensor)
        val_loss = criterion(val_outputs, y_test_tensor)
        val_losses.append(val_loss.item())
    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
    pbar.update(1)
    pbar.set_postfix({"Training Loss": train_losses[-1], "Validation Loss": val_losses[-1]})

#### Evaluate the model
Again, take careful note of this code - you will be using it.

In [ ]:
# Evaluate the model
ann1_model.eval()
with tch.no_grad():
    y_pred = ann1_model(X_test_tensor)
    y_pred_np = y_pred.cpu().detach().numpy()
    y_test_np = y_test_tensor.cpu().detach().numpy()
    ann_mse = mean_squared_error(y_test_np, y_pred_np)
    print('ANNa Regression: MSE = {:.3f}'.format(ann_mse))

# Plot training history
plot_history(train_losses, val_losses, 'ANN1 Regression')

## Going deeper with ANNs 
In the cells below we define, compile, fit and evaluate a neural network model with:
* two hiiden layers, each with 32 neurons and non-linear activations (relu)
* a dropout layer at the end with a dropout rate of 0.2

<p>
    <img src="figs/relu-activation.png" alt="drawing" style="width:500px;"/>
    <center>Figure 2. relu activation.</center>
</p>

Dropout can help to avoid overfitting, much as L1 and L2 regularizations do (as you explored in the supervise machine learning lab). In the model loss plots (below) this stops the test loss from increasing as you train for more epochs.

Some quotes from a paper co-authored by members of our group called "Deep Learning in Image Cytometry: A Review" (https://onlinelibrary.wiley.com/doi/full/10.1002/cyto.a.23701):

"_Overfitting occurs when the parameters of a model fit too closely to the input training data, without capturing the underlying distribution, and thus reducing the model’s ability to generalize to other datasets_".

DROPOUT: "_A regularization technique that reduces the interdependent learning among the neurons to prevent overfitting. Some neurons are randomly “dropped,” or disconnected from other neurons, at every training iteration, removing their influence on the optimization of the other neurons. Dropout creates a sparse network composed of several networks—each trained with a subset of the neurons. This transformation into an ensemble of networks hugely decreases the possibility of overfitting, and can lead to better generalization and increased accuracy_".

<p>
    <img src="figs/dropout.png" alt="drawing" style="width:1200px;"/>
    <center>Figure 3. Dropout.</center>
</p>



In [ ]:
class ANN2(nn.Module):
    def __init__(self, input_dim):
        super(ANN2, self).__init__()
        ### Add Code here
    
    def forward(self, x):
        ### Add code here
        return x

In [ ]:
# Initialize the ANN2 model and summmarize it
# input_dim = 
# asummary()

# Loss and optimizer
# criterion = 
# optimizer = 



In [ ]:
num_epochs = 100
train_losses = []
val_losses = []

pbar = tqdm(range(num_epochs), desc="Epochs")



for epoch in range(num_epochs):
    
    pbar.update(1)
    pbar.set_postfix({"Training Loss": train_losses[-1], "Validation Loss": val_losses[-1]})

In [ ]:
# Evaluate the model
# ann2_model.eval()
# with tch.no_grad():


# Plot training history
plot_history(train_losses, val_losses, 'ANN2 Regression')

### Testing GNNs
So, now you've tested regression on molecular descriptors with ANNs. ANother option that is gaining traction in the research world is using Graph Neural Networks or, as they can also be called, Graph Convolutional Networks. You will be using an extension library called Pytorch.Geometric for this. Training and Evaluation looks the same.

In [ ]:
class GNN1(nn.Module):
    def __init__(self, input_dim):
        super(GNN1, self).__init__()
        self.conv1 = GCNConv(input_dim, 32)
        self.conv2 = GCNConv(32, 16)
        self.fc3 = nn.Linear(16, 1)  # Output layer with 1 node
        self.relu = nn.ReLU()

    def forward(self, x, edge_index, batch):
        # x, edge_index = data.x, data.edge_index
        x = self.relu(self.conv1(x, edge_index))
        x = self.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc3(x)
        return x
    

In [ ]:
# Initialize the GNN1 model
input_dim = data_train.iloc[5].x.size(1)
gnn1_model = GNN1(input_dim)
#Loss and optimizer
num_epochs = 100
batch_size = 8
criterion = nn.MSELoss()
optimizer = optim.Adam(gnn1_model.parameters(), lr=0.001)  # Adjust learning rate as needed
t_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
v_loader = DataLoader(data_test, batch_size=batch_size, shuffle=True)


In [ ]:
#Fit GNN Model here. Hint, this is a bit harder - you need to account for batches, and averages. You have some more help-code prepared. Fix the commented sections. Take inspiration from your previous training-loop

train_losses = []
val_losses = []
pbar = tqdm(range(num_epochs), desc="Epochs")
for epoch in range(num_epochs):
    # Training Phase. Don't forget to save you losses! and to set your model to train!
    # gnn1_model.train()

    for batch in t_loader:
        optimizer.zero_grad()
        # Use Batch Data object in forward pass
        outputs = gnn1_model(batch.x, batch.edge_index, batch.batch)
        #Here you need some code. it should look very similar to you previous loop for ANN2


    avg_train_loss = sum(train_loss_items) / len(train_loss_items)
    train_losses.append(avg_train_loss)
    # Validation Phase (assuming you have a separate validation loader)
    gnn1_model.eval()
    val_loss_items = []
    with tch.no_grad():
        for val_batch in v_loader:
            #here you run your validation per batch
 
    # I suggest you deal with val_loss as you did with train_loss
    val_losses.append(avg_val_loss)
    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_losses[-1]:.4f}, Val Loss: {avg_val_loss:.4f}')
    pbar.update(1)
    pbar.set_postfix({"Training Loss": avg_train_loss, "Validation Loss": avg_val_loss})



### We've prepared the eval for you.
If you did everything else correctly, this will run out-of-the-box

In [ ]:
gnn1_model.eval()
all_preds = []
all_labels = []
with tch.no_grad():
    for v_batch in v_loader:
        test_outputs = gnn1_model(v_batch.x, v_batch.edge_index, v_batch.batch)
        all_preds.extend(test_outputs.tolist())
        all_labels.extend(v_batch.y.tolist())
all_preds_tensor = tch.tensor(all_preds)
all_labels_tensor = tch.tensor(all_labels)
gnn1_mse = mean_squared_error(all_labels_tensor, all_preds_tensor)

print(f'GNN1 Regression: MSE = {gnn1_mse:.3f}')
plot_history(train_losses, val_losses, 'GNN1 Regression')